# 3.1.6 Exercises


##### 1. Assume that we have some data $x_1, \ldots, x_n \in \mathbb{R}$. Our goal is to find a constant $b$ such that $\sum_i (x_i - b)^2$ is minimized.

1. Find an analytic solution for the optimal value of $b$.
2. How does this problem and its solution relate to the normal distribution?
3. What if we change the loss from $\sum_i (x_i - b)^2$ to $\sum_i |x_i-b|$? Can you find the optimal solution for $b$?

假设我们有一些数据 $x_1, \ldots, x_n \in \mathbb{R}$。我们的目标是找到一个常数 $b$，使得 $\sum_i (x_i - b)^2$ 最小化。

1. 找到 $b$ 的最优值的解析解。
2. 这个问题及其解决方案与正态分布有什么关系？
3. 如果我们将损失从 $\sum_i (x_i - b)^2$ 更改为 $\sum_i |x_i-b|$? 会怎样？你能找到 $b$ 的最优解吗？


A. $b$ 的最优值的解析解公式如下：

$$\sum_i (x_i-b)^2 = \sum_i x_i^2 - 2b\sum_i x_i + nb^2=n(b-\frac{1}{n}\sum_i x_i)^2+\sum_i x_i^2-\frac{(\sum_i x_i)^2}{n}$$
让 $b = \frac{1}{n}\sum_i x_i$，函数将获得最小值。

B.

假设随机变量 $X = b + \epsilon$ ，其中 𝜖 正态分布的噪声, 如果我们想估计参数 $b$, 则通过 $\sum_i (x_i - b)^2$（最小化所有样本的平方误差和）， 最终的解是所有样本的均值。

C.
From the [link](https://math.stackexchange.com/questions/113270/the-median-minimizes-the-sum-of-absolute-deviations-the-ell-1-norm)

![3_1_1](material/3_1_1.png)


##### 2. Prove that the affine functions that can be expressed by $\mathbf{x}^\top \mathbf{w} + b$ are equivalent to linear functions on $(\mathbf{x}, 1)$.

证明可以用 $\mathbf{x}^\top \mathbf{w} + b$ 表示的仿射函数与 $(\mathbf{x}, 1)$ 是等价的。


解：

$$
\mathbf{x}^T\mathbf{w}+b=x_1*w_1 + ... + x_n * w_n + 1 * b = (\mathbf{x},1)^T(\mathbf{w},b)
$$

$(\mathbf{x},1)$ 相当于扩充 1 列（值全为 1），这样 $\mathbf{x}$ 就可以直接和 $\mathbf{w}$ 将 $b$ 扩充后的 $\mathbf{w}$ 相乘。


##### 3. Assume that you want to find quadratic functions of $\mathbf{x}$, i.e., $f(\mathbf{x}) = b + \sum_i w_i x_i + \sum_{j \leq i} w_{ij} x_{i} x_{j}$. How would you formulate this in a deep network?

**假设你想找到 $\mathbf{x}$ 的二次函数，既 $f(\mathbf{x}) = b + \sum_i w_i x_i + \sum_{j \leq i} w_{ij} x_{i} x_{j}$。你将如何在深度网络中表述这一点？**


In [1]:
import torch
import torch.nn as nn


# 计算输入向量 x 的所有元素对的乘积，并将结果存储在一个一维张量中。
def mul(x):
  n = x.size(0)
  res = torch.zeros(int(n * (n + 1) / 2))
  index = 0
  for i in range(0, n):
    for j in range(i, n):
      res[index] = x[i] * x[j]
      index += 1
  return res


class Model(nn.Module):
  def __init__(self, n):
    super(Model, self).__init__()
    # 定义两个全连接层
    self.fc1 = nn.Linear(n, 1)
    self.fc2 = nn.Linear(int(n * (n + 1) / 2), 1)

  def forward(self, x):
    y1 = self.fc1(x)
    y2 = mul(x)
    y2 = self.fc2(y2)
    y = y1 + y2
    return y


model = Model(10)
x = torch.randn(10)
y = model(x)
print(x)
print(y)

tensor([-1.1457,  0.6503, -0.8702,  1.3687, -0.5948,  0.0442,  0.6812, -2.2838,
        -0.5610,  0.0743])
tensor([1.2569], grad_fn=<AddBackward0>)


##### 4. Recall that one of the conditions for the linear regression problem to be solvable was that the design matrix $\mathbf{X}^\top \mathbf{X}$ has full rank.

1. What happens if this is not the case?
2. How could you fix it? What happens if you add a small amount of coordinate-wise independent Gaussian noise to all entries of $\mathbf{X}$?
3. What is the expected value of the design matrix $\mathbf{X}^\top \mathbf{X}$ in this case?
4. What happens with stochastic gradient descent when $\mathbf{X}^\top \mathbf{X}$ does not have full rank?

**请记住，线性回归问题可解的条件之一是设计矩阵 $\mathbf{X}^\top \mathbf{X}$ 具有满秩。**

1. 如果情况不是这样，会发生什么？
2. 你怎么能修复它？如果你在 $\mathbf{X}$ 的所有条目中添加少量坐标独立的高斯噪声，会发生什么？
3. 在这种情况下，设计矩阵 $\mathbf{X}^\top \mathbf{X}$ 的期望值是多少？
4. 当 $\mathbf{X}^\top \mathbf{X}$ 不具有满秩时，随机梯度下降会发生什么？


答：

1. 线性回归问题可能无解或解不唯一。
2. 可以通过正则化（如岭回归）或添加高斯噪声来使 $\mathbf{X}^\top \mathbf{X}$ 具有满秩，从而解决该问题。
3. 添加高斯噪声后，$\mathbf{X}^\top \mathbf{X}$ 的期望值会接近一个正定矩阵，从而确保其具有满秩。
4. 随机梯度下降法（SGD）可能会陷入局部最优解或收敛速度变慢。SGD $(\mathbf{X}^\top \mathbf{X})$ 不为满秩时仍然可以使用，但可能无法收敛到唯一解。这是因为 SGD 不依赖于 $(\mathbf{X}^\top \mathbf{X})$ 的可逆性，而是基于随机选择的数据子集迭代更新回归系数。然而，$(\mathbf{X})$ 中线性相关列的存在可能导致损失面具有多个最小值，这意味着 SGD 可能会根据回归系数的初始值收敛到不同的解。


##### 5. Assume that the noise model governing the additive noise $\epsilon$ is the exponential distribution. That is, $p(\epsilon) = \frac{1}{2} \exp(-|\epsilon|)$.

1. Write out the negative log-likelihood of the data under the model $-\log P(\mathbf y \mid \mathbf X)$.
2. Can you find a closed form solution?
3. Suggest a minibatch stochastic gradient descent algorithm to solve this problem. What could possibly go wrong (hint: what happens near the stationary point as we keep on updating the parameters)? Can you fix this?

假设控制加性噪声 $\epsilon$ 的噪声模型是指数分布。也就是说 $p(\epsilon) = \frac{1}{2} \exp(-|\epsilon|)$ 。

1. 在模型 $-\log P(\mathbf y \mid \mathbf X)$ 下写出数据的负对数似然。
2. 你能找到一个封闭形式的解吗？
3. 建议一个小批量随机梯度下降算法来解决这个问题。可能会出现什么问题（提示：当我们不断更新参数时，靠近驻点会发生什么）？你能解决这个问题吗？


答：

1. $$P(\mathbf{y}|\mathbf{X})=P(\epsilon=y-Xw)=\prod \frac{1}{2}exp(-|y_i-x_i^Tw|)$$
   $$L = -logP(\mathbf{y}|\mathbf{X})=Nlog2+\sum_i \|y_i-x_i^Tw\|$$

2. 找不到封闭形式的解。

$$\bigtriangledown_w(L)=-\sum sign(y_i-xi^Tw)x_i=0$$

3. 因此，梯度不是一个平滑的函数，并且在 $sign(y_i-xi^Tw)$ 改变符号的地方有跳跃。因此，该函数将很难收敛。我们可以使用更大的批量大小。


##### 6. Assume that we want to design a neural network with two layers by composing two linear layers. That is, the output of the first layer becomes the input of the second layer. Why would such a naive composition not work?

设我们想通过组合两个线性层来设计一个具有两层的神经网络。也就是说，第一个层的输出成为第二个层的输入。为什么这样的简单组合不起作用？


答：

在神经网络中，两个线性层的组合本质上会导致一个线性层。这是由于线性性质：两个线性函数的组合是另一个线性函数。

这样，无法引入非线性，从而限制了模型的表达能力。


##### 7. What happens if you want to use regression for realistic price estimation of houses or stock prices?

1. Show that the additive Gaussian noise assumption is not appropriate. Hint: can we have negative prices? What about fluctuations?
2. Why would regression to the logarithm of the price be much better, i.e., $y = \log \textrm{price}$?
3. What do you need to worry about when dealing with pennystock, i.e., stock with very low prices? Hint: can you trade at all possible prices? Why is this a bigger problem for cheap stock? For more information review the celebrated Black--Scholes model for option pricing

**如果您想使用回归来对房屋或股票价格进行实际价格估算，会发生什么？**

1. 证明加性高斯噪声假设不合适。提示：我们能有负价格吗？波动呢？
2. 为什么对价格的对数进行回归会更好，即 $y = \log \textrm{price}$ ?
3. 在处理便士股票（即价格非常低的股票）时，您需要担心什么？提示：您能否在所有可能的价格进行交易？为什么这对便宜股票来说是一个更大的问题？有关更多信息，请查看著名的布莱克-斯科尔斯期权定价模型。


答：

1. 价格不能为负，但高斯噪声允许出现负值。股票价格波动与其历史记录相关。
2. $y$ 的范围是整个实数域。
3. 如果价格变得非常微小，其对数将变为一个非常大的负数，这会导致价值剧烈变化。这些股票的价格可能无法平稳变化，因为最小变动单位或价格变化的最小增量。这可能导致价格变化的分布不连续，而高斯分布无法很好地对其建模。


##### 8. Suppose we want to use regression to estimate the _number_ of apples sold in a grocery store.

1. What are the problems with a Gaussian additive noise model? Hint: you are selling apples, not oil.
2. The [Poisson distribution](https://en.wikipedia.org/wiki/Poisson_distribution) captures distributions over counts. It is given by $p(k \mid \lambda) = \lambda^k e^{-\lambda}/k!$. Here $\lambda$ is the rate function and $k$ is the number of events you see. Prove that $\lambda$ is the expected value of counts $k$.
3. Design a loss function associated with the Poisson distribution.
4. Design a loss function for estimating $\log \lambda$ instead.

**假设我们想使用回归来估计杂货店销售的苹果数量。**

1. 高斯加性噪声模型存在哪些问题？提示：你在卖苹果，而不是石油。
2. [泊松分布](https://en.wikipedia.org/wiki/Poisson_distribution) 捕捉计数的分布。它由 $p(k \mid \lambda) = \lambda^k e^{-\lambda}/k!$ 给出。这里 $\lambda$ 是速率函数， $k$ 是你看到的事件数量。证明 $\lambda$ 是计数 $k$ 的期望值。
3. 设计与泊松分布相关的损失函数。
4. 设计一个损失函数来估计 $\log \lambda$。


答：

1. 苹果的数量是一个整数，因此是离散的。但高斯加性噪声是连续的。
2. $$E(k)=\sum_{k=1}^\infty k \frac{\lambda^k e^{-\lambda}}{k!}=\lambda e^{-\lambda}\sum_{k=1}^\infty \frac{\lambda^{k-1}}{(k-1)!}=\lambda e^{-\lambda}\sum_{k=0}^\infty \frac{\lambda^{k}}{(k)!}=\lambda e^{-\lambda} e^{\lambda}=\lambda$$
3. $$-log(P(K|\lambda))=-\sum_i (k_ilog\lambda -\lambda - log(k_i!))= n\lambda - log\lambda \sum_i k_i + \sum_i (log(k_i!))$$
   $\lambda$ 的损失函数是：
   $$L(\lambda)=n\lambda - log\lambda \sum_i k_i$$
4. 让 $t = log\lambda$
   $$L(t)=ne^t - t \sum_i k_i$$
